# WiDS Week 2 Project - Nikhil Kocherry

In [3]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import itertools
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import yfinance as yf
from tensorflow.keras.callbacks import EarlyStopping
from scalecast.Forecaster import Forecaster
warnings.filterwarnings('ignore')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 1. Data Collection

I imported stock information of 'AAPL' from yfinance (just as was done in the previous project).

## 2. Data Preprocessing

I used the exact same approach as in the previous project. The test-train split has been done seaparately for both models.

In [4]:
dat = yf.download("AAPL",period = "max", progress = "False")
dat.columns = dat.columns.droplevel(1)
dat = dat.dropna()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: SSLError('Failed to perform, curl: (35) Recv failure: Connection reset by peer. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


## 3. ARIMA Model

This is an exact copy of last project, barring the creation of a Forecaster object 'f', in order to compare both the models better.

In [5]:
f = Forecaster(y = dat['Close'],current_dates = dat.index)
f.set_test_length(0.2)

p = range(0, 4)
d = range(0, 3)
q = range(0, 4)
pdq = list(itertools.product(p, d, q))

best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = ARIMA(f, order=order)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except:
        continue
        
f.set_estimator('arima',order = best_order)

ValueError: Need at least 3 dates to infer frequency

## 4. LSTM Model

We are registering the LSTM model to the Forecaster object using the manual_forecast() function, as this estimator is not pre-defined in ScaleCast.

In [ ]:
f.manual_forecast(
    call_me='lstm_best',
    lags=36,
    batch_size=32,
    epochs=15,
    validation_split = 0.2
    shuffle = False
    activation='tanh',
    optimizer='Adam',
    learning_rate=0.001,
    lstm_layer_sizes=(72,)*4,
    dropout=(0,)*4,
    plot_loss=False
)

Both the models are now fitted to the time series stored in the Forecaster object 'f', and the results of the models are plotted against the actual prices of the test data.

In [1]:
f.fit()
f.predict()
f.plot(models = ['arima','lstm_best'])

NameError: name 'f' is not defined

## 5. Comparison of ARIMA and LSTM

The following statement gives us all the required error data of the two models, i.e. Mean Absolute Error, Root Mean absolute Error, and Mean Abspolute Percentage Error, in the form of a table.

In [ ]:
f.evaluate()

## 7. Residual Comparison

The following statements give us the plot of the error of the two models, in the form of their residue plot.

In [ ]:
f.plot(f.y_test - f.forecasts['arima']['forecast'] , label = 'ARIMA_Residual')
f.plot(f.y_test - f.forecasts['lstm_best']['forecast'] , label = 'LSTM_Residual')

# And the concludes the Assignment for Week 2!!